# Projeto 1 - Ciência dos Dados

Nome: Alison Araujo

Nome: Gabrielly Carneiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [18]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('word_tokenize')
stopwordsdic = stopwords.words('portuguese')

from spacy import load
nlp = load('pt_core_news_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


In [19]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Projeto 1\C.DadosP1


Carregando a base de dados com os tweets classificados manualmente:

In [20]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Acionável (para amazon)/direcionável (para editora)/não acionável(para o autor ou irrelevante)
0,"Para quem gosta de poemas simples, esse é o li...",N
1,Quando vi o lançamento pensei que finalmente p...,D
2,"É incrível como esses escritores e ""intelectua...",N
3,"Se desse pra devolver eu devolvia, nao é por e...",N
4,"Futilidade, inutilidade, desperdício de papel,...",N


In [21]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Acionável (para amazon)/direcionável (para editora)/não acionável(autor ou irrelevante)
0,"O livro é prolixo, redundante, doentio. Sou su...",N
1,"Bom livro e história envolvente. Porém, o leit...",D
2,"Fala tudo que todos já sabem, sem falar que nã...",N
3,Ganhei na compra do Kindle. Não é estilo de li...,N
4,"Custa crer que um livro tão medíocre, embora m...",N


___
## Classificador automático


Na primeira etapa, de classificação manual, consideramos três targets para os reviews: Acionável, Direcionável e Não Acionável. 

- Acionável: para ser considerado "acionável" ("A") o review deve ser passível de alguma ação pela Amazon, ou seja, o review deve ser sobre entrega, estado do produto, contato com o suporte, etc. 
- Direcionável: para o target "direcionável" ("D") foram considerados comentários relativos à editora, como qualidade do material do livro, preço do livro e do e-book, tradução e edição. 
- Não Acionáveis: por fim, os não acionáveis ("NA") eram comentários relativos ao autor, ao apreço pelo conteúdo do livro, ou comentários irrelavantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [22]:
#Criar uma função que transforma as frases da planilha em um texto só 
    #(Será útil para criar o dicionário com as palavras)
def transforma_em_string(coluna):
    texto = ''
    for linha in coluna:
        texto += linha + ' '
    return texto    

In [23]:
#Criar uma função que limpa todas as pontuações
def cleanup(text):
    punctuation = r'[´"\'!-.:?;$,/~^_=+*&¨%$#@|\{}()[\]]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [24]:
#Cria uma função que limpa os espaços duplicados
def limpa_espaco(text):
    punctuation = r'[\n]'  # Adicione os caracteres desejados aqui
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [25]:
#Criando uma função para remover emoji
def remove_emoji(text):
    text_without_emojis = unidecode(text)
    return text_without_emojis

In [26]:
#Criando a função de stopwords
def stopwords(texto):
    palavras = word_tokenize(texto, language='portuguese') # Tokenize é analisar palavras individualmente, basicamente
    palavras_sem_stopword = []
    for palavra in palavras:
        if palavra not in stopwordsdic:
            palavras_sem_stopword.append(palavra)
    # Reúna as palavras sem stopwords em uma string novamente
    texto_sem_stopword = ' '.join(palavras_sem_stopword)
    return texto_sem_stopword

In [27]:
#Criando a função de lematização
def lemmat(texto):
    doc = nlp(texto)
    lemmat_radicais = []
    for radicais in doc:
        lemmat_radicais.append(radicais.lemma_)
    texto_lemmat = ' '.join(lemmat_radicais)    
    return texto_lemmat

In [28]:
# Cria uma função que reúna as funções de limpeza
def limpa_tudo(mensagem):
    #Aplicando a função de limpeza de pontuação
    texto = cleanup(mensagem)
    #Deixando tudo em letra minúscula
    texto = texto.lower()
    #Removendo emoji
    texto = remove_emoji(texto)
    #Aplicando a função de limpeza de espaço
    texto = limpa_espaco(texto)
    #Removendo stopwords
    texto = stopwords(texto)
    #Realiza lemmatização
#     texto = lemmat(texto)
    return texto


In [29]:
#Cria uma função que limpa as linhas da planilha e adiciona as mensagens limpas à planilha
def mensagem_limpa(planilha):   #recebe a planilha e retorna com a coluna de mensagem limpa
    planilha['Mensagem Limpa'] = [limpa_tudo(x) for x in list(planilha['Mensagem'])]
    return planilha

In [30]:
#Cria dicionário de tudo
def cria_vocabulario(mensagem):
    lista_palavras = transforma_em_string(mensagem)
    lista_palavras = limpa_tudo(lista_palavras)
    lista_palavras = lista_palavras.split()
    return lista_palavras     #devolve uma lista com as palavras separadas

In [31]:
#Cria função que retorna uma lista sem as palavras repetidas
def remove_repeticao(lista):
    dic = set(lista)
    vocabulario = list(dic)
    return vocabulario

In [32]:
# Cria uma função que guarda as palavras em pd.Series
def cria_pdseries(lista):
    tabela = pd.Series(lista)
    return tabela

#Cria uma função que retorna a frequência absoluta de cada palavra no texto
def freq_abs(tabela):
    absoluta = tabela.value_counts()
    return absoluta

#Cria uma função que retorna a frequência relativa de cada palavra no texto
def freq_rel(tabela):
    relativa = tabela.value_counts(True)
    return relativa

In [36]:
################# TESTES #################

vocabulario = cria_vocabulario(train.Mensagem)
vocabulario = remove_repeticao(vocabulario)
df_vocabulario = cria_pdseries(vocabulario)

## nessa etapa o que eu fiz foi criar um vocabulário de todas as palavras contidas nas mensagens

In [37]:
####-  DIVISÃO CATEGORIAS TREINO  -####

#--- categoria Não Acionável
train_N = train.loc[train['Acionável (para amazon)/direcionável (para editora)/não acionável(para o autor ou irrelevante)'] == 'N']

vocab_train_N = cria_vocabulario(train_N.Mensagem)
    #vocab_train_N = remove_repeticao(vocab_train_N)

df_vocab_train_N = cria_pdseries(vocab_train_N)

freq_rel_train_N = freq_rel(df_vocab_train_N)
freq_abs_train_N = freq_abs(df_vocab_train_N)

total_train_N = freq_abs_train_N.sum()

#--- categoria Direcionável
train_D = train.loc[train['Acionável (para amazon)/direcionável (para editora)/não acionável(para o autor ou irrelevante)'] == 'D']

vocab_train_D = cria_vocabulario(train_D.Mensagem)
    #vocab_train_D = remove_repeticao(vocab_train_D)

df_vocab_train_D = cria_pdseries(vocab_train_D)

freq_rel_train_D = freq_rel(df_vocab_train_D)
freq_abs_train_D = freq_abs(df_vocab_train_D)

total_train_D = freq_abs_train_D.sum()

#--- categoria Acionável
train_A = train.loc[train['Acionável (para amazon)/direcionável (para editora)/não acionável(para o autor ou irrelevante)'] == 'A']

vocab_train_A = cria_vocabulario(train_A.Mensagem)
    #vocab_train_A = remove_repeticao(vocab_train_A)

df_vocab_train_A = cria_pdseries(vocab_train_A)

freq_rel_train_A = freq_rel(df_vocab_train_A)
freq_abs_train_A = freq_abs(df_vocab_train_A)

total_train_A = freq_abs_train_A.sum()

In [38]:
####-  DIVISÃO CATEGORIAS TESTE  -####

#--- categoria Não Acionável
test_N = test.loc[test['Acionável (para amazon)/direcionável (para editora)/não acionável(autor ou irrelevante)'] == 'N']

vocab_test_N = cria_vocabulario(test_N.Mensagem)
    #vocab_test_N = remove_repeticao(vocab_test_N)

df_vocab_test_N = cria_pdseries(vocab_test_N)

freq_rel_test_N = freq_rel(df_vocab_test_N)
freq_abs_test_N = freq_abs(df_vocab_test_N)

total_test_N = freq_abs_test_N.sum()

#--- categoria Direcionável
test_D = test.loc[test['Acionável (para amazon)/direcionável (para editora)/não acionável(autor ou irrelevante)'] == 'D']

vocab_test_D = cria_vocabulario(test_D.Mensagem)
    #vocab_test_D = remove_repeticao(vocab_test_D)

df_vocab_test_D = cria_pdseries(vocab_test_D)

freq_rel_test_D = freq_rel(df_vocab_test_D)
freq_abs_test_D = freq_abs(df_vocab_test_D)

total_test_D = freq_abs_test_D.sum()

#--- categoria Acionável
test_A = test.loc[test['Acionável (para amazon)/direcionável (para editora)/não acionável(autor ou irrelevante)'] == 'A']

vocab_test_A = cria_vocabulario(test_A.Mensagem)
    #vocab_test_A = remove_repeticao(vocab_test_A)

df_vocab_test_A = cria_pdseries(vocab_test_A)
freq_rel_test_A = freq_rel(df_vocab_test_A)
freq_abs_test_A = freq_abs(df_vocab_test_A)

total_test_A = freq_abs_test_A.sum()

total_test_A + total_test_D + total_test_N


7419

In [39]:
####- TREINO TODAS AS PALAVRAS -####
vocab_train = cria_vocabulario(train.Mensagem)
#vocab_train = remove_repeticao(vocab_train)

df_vocab_train = cria_pdseries(vocab_train)

freq_rel_train = freq_rel(df_vocab_train)
freq_abs_train = freq_abs(df_vocab_train)

total_train = freq_abs_train.sum()
total_train

15675

In [40]:
####- TESTE TODAS AS PALAVRAS -####

vocab_test = cria_vocabulario(test.Mensagem)
#vocab_train = remove_repeticao(vocab_train)

df_vocab_test = cria_pdseries(vocab_test)

freq_rel_test = freq_rel(df_vocab_test)
freq_abs_test = freq_abs(df_vocab_test)

total_test = freq_abs_test.sum()
total_test


7419

In [41]:
'''EXPLICAÇÕES
vocab_df [lista] = palavras já limpas

df_vocab_df [tabela] = transforma o vocab_df em tabela

freq_rel_df [tabela] = frequencia relativa de cada palavra
freq_abs_df [tabela] = 1
total_df [int] = quantidade total de palavras
'''

'EXPLICAÇÕES\nvocab_df [lista] = palavras já limpas\n\ndf_vocab_df [tabela] = transforma o vocab_df em tabela\n\nfreq_rel_df [tabela] = frequencia relativa de cada palavra\nfreq_abs_df [tabela] = 1\ntotal_df [int] = quantidade total de palavras\n'

In [42]:
####- PROBABILIDADES DAS CATEGORIAS TESTE -####
prob_test_N = total_test_N/total_test
prob_test_D = total_test_D/total_test
prob_test_A = total_test_A/total_test

prob_test_A + prob_test_D + prob_test_N

1.0

In [43]:
####- PROBABILIDADES DAS CATEGORIAS TREINO -####
prob_trein_N = total_train_N/total_train
prob_trein_D = total_train_D/total_train
prob_trein_A = total_train_A/total_train

prob_trein_N + prob_trein_A + prob_trein_D

1.0

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

In [ ]:
#a probabilidade de todas as palavras estarem contidas num texto não é a mesma? 

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Dica: apresentar um grafico com testes mostrando quais limpezas melhoraram a acurácia do nosso classificador (lemmatization, stopwords, etc)